In [5]:
#sf api setup

import time
import asyncio
import logging

from os.path import join

from sfapi_client         import Client, AsyncClient
from sfapi_client.compute import Machine
from sfapi_client.jobs    import JobState

from sfapi_connector import KeyManager, OsSFAPI, OsWrapper, LOGGER




In [6]:
# sf api send jobscript

N = 10000

target = "./sfapi_test"


    
job_script = f"""#!/bin/bash
#SBATCH -q debug
#SBATCH -A m669
#SBATCH -N 2
#SBATCH -n 64              # Number of tasks (64 tasks, 32 per node)
#SBATCH -C cpu
#SBATCH -t 00:01:00
#SBATCH -J sfapi-demo
#SBATCH --exclusive
#SBATCH --output={target}/sfapi-demo-%j.out
#SBATCH --error={target}/sfapi-demo-%j.error


# Load necessary modules, for example Python and Dask
module load python dask

cd $SCRATCH/nersc-notebooks/perlmutter/dask

& ./launch.sh


"""


# job_script = f"""#!/bin/bash
# #SBATCH -q debug
# #SBATCH -A m669
# #SBATCH -N 1
# #SBATCH -C cpu
# #SBATCH -t 00:01:00
# #SBATCH -J sfapi-demo
# #SBATCH --exclusive
# #SBATCH --output={target}/sfapi-demo-%j.out
# #SBATCH --error={target}/sfapi-demo-%j.error

# module load python
# # Prints N random numbers to form a normal distrobution
# python -c "import numpy as np; numbers = np.random.normal(size={N}); [print(n) for n in numbers]"
#     """ 

# os = OsWrapper()
# job_stript_path = join(target, "job_script.sh")

# with os.open(job_stript_path, "w", mk_target_dir=False) as f:
#     f.write(job_script)

km = KeyManager()
with Client(key=km.key) as client:
    perlmutter = client.compute(Machine.perlmutter)


    job = perlmutter.submit_job(job_script)
    print(f"Submitted_job: {job.jobid}")
    job_id = job.jobid

    while True:
        job.update()
        print(f"The job state is: {job.state} ({type(job.state)})")
        if job.state not in [JobState.PENDING, JobState.RUNNING, JobState.COMPLETING]:
            if job.state == JobState.FAILED:
                print("Job failed")
            elif job.state == JobState.COMPLETED:
                print("Job completed")
                output_file = perlmutter.ls(f"/global/homes/s/sanjeevc/sfapi_test/sfapi-demo-{job_id}.out")
                output_file = output_file[0]
                with output_file.open("r") as f:
                    print(f.read())
            elif job.state == JobState.TIMEOUT:
                print("Job timeout")
                output_file = perlmutter.ls(f"/global/homes/s/sanjeevc/sfapi_test/sfapi-demo-{job_id}.out")
                output_file = output_file[0]
                with output_file.open("r") as f:
                    print(f.read())    
            break
        time.sleep(10)


HTTPStatusError: Client error '403 Forbidden' for url 'https://api.nersc.gov/api/v1.2/account/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403

In [5]:
output_file = perlmutter.ls(f"{target}/*.out")
output_file = output_file[0]
with output_file.open("r") as f:
    print(f.read())


RuntimeError: Cannot send a request, as the client has been closed.